In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DATA_DIR = "/content/drive/MyDrive/Bark images 2"
SWEEP_DIR = "/content/drive/MyDrive/bark_sweeps/resnet50_sweep"


Mounted at /content/drive


## SETUP

In [ ]:
import random
from pathlib import Path
import os, random
import numpy as np
import torch

SEED = 42

def seed_everything(seed=SEED):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # Deterministic (slower, but reproducible)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)
print("Seed set to:", SEED)

def list_samples_grouped_by_tree(root_dir, exts={".jpg",".jpeg",".png",".bmp",".webp"}):
    root = Path(root_dir)
    classes = sorted([p.name for p in root.iterdir() if p.is_dir()])
    class_to_idx = {c:i for i,c in enumerate(classes)}

    samples = []  # (img_path, label_idx, group_id)
    for cls in classes:
        cls_dir = root / cls
        tree_ids = sorted([p.name for p in cls_dir.iterdir() if p.is_dir()])
        for tree_id in tree_ids:
            tree_dir = cls_dir / tree_id
            group_id = f"{cls}/{tree_id}"  # unique per tree within class
            for img_path in tree_dir.rglob("*"):
                if img_path.is_file() and img_path.suffix.lower() in exts:
                    samples.append((str(img_path), class_to_idx[cls], group_id))
    return samples, classes, class_to_idx

def split_by_tree_within_each_class(samples, test_ratio=0.2, seed=42):
    rng = random.Random(seed)

    by_class = {}
    for path, y, group in samples:
        by_class.setdefault(y, {}).setdefault(group, []).append((path, y, group))

    train, test = [], []
    for y, groups_dict in by_class.items():
        groups = list(groups_dict.keys())
        rng.shuffle(groups)

        n_test = max(1, int(round(len(groups) * test_ratio))) if len(groups) > 1 else 0
        test_groups = set(groups[:n_test])

        for g, items in groups_dict.items():
            (test if g in test_groups else train).extend(items)

    rng.shuffle(train)
    rng.shuffle(test)
    return train, test

samples, class_names, class_to_idx = list_samples_grouped_by_tree(DATA_DIR)
train_samples, test_samples = split_by_tree_within_each_class(samples, test_ratio=0.2, seed=42)

# Leak check
train_groups = {g for _,_,g in train_samples}
test_groups  = {g for _,_,g in test_samples}
assert train_groups.isdisjoint(test_groups), "Leakage: same tree folder is in both train and test!"

print("Classes:", class_names)
print("Train images:", len(train_samples), "Test images:", len(test_samples))


Seed set to: 42
Classes: ['acacia', 'apple', 'candle', 'cherry', 'cypress', 'fig', 'katsura', 'linden', 'magnolia', 'oak', 'olive', 'palm', 'pine', 'plane', 'red-gum', 'sakura']
Train images: 870 Test images: 256


### MODEL AND CHECKPOINTS


In [ ]:
from PIL import Image
from torch.utils.data import Dataset

class BarkDataset(Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, y, _group = self.samples[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, y


In [ ]:
import torch

def save_checkpoint(path, *, epoch, model, optimizer, scheduler=None, scaler=None,
                    best_metric=None, class_names=None, extra=None):
    ckpt = {
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "best_metric": best_metric,
        "class_names": class_names,
        "extra": extra or {},
        "torch_version": torch.__version__,
    }
    if scheduler is not None:
        ckpt["scheduler_state"] = scheduler.state_dict()
    if scaler is not None:
        ckpt["scaler_state"] = scaler.state_dict()
    torch.save(ckpt, path)


In [ ]:
import os, json, time
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader
from pathlib import Path

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

def make_loaders(input_size, batch_size=32):
    train_tfms = transforms.Compose([
        transforms.Resize(input_size),
        transforms.RandomCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(0.2, 0.2, 0.2, 0.05),
        transforms.ToTensor(),
    ])
    test_tfms = transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
    ])

    train_ds = BarkDataset(train_samples, transform=train_tfms)
    test_ds  = BarkDataset(test_samples,  transform=test_tfms)

    if input_size >= 384:
        batch_size = min(batch_size, 16)
    if input_size >= 450:
        batch_size = min(batch_size, 8)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=2, pin_memory=True)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    return train_loader, test_loader

def make_optimizer(name, params, lr, weight_decay, momentum=0.9):
    name = name.lower()
    if name == "adamw":
        return torch.optim.AdamW(params, lr=lr, weight_decay=weight_decay)
    if name == "adam":
        return torch.optim.Adam(params, lr=lr, weight_decay=weight_decay)
    if name == "sgd":
        return torch.optim.SGD(params, lr=lr, momentum=momentum, weight_decay=weight_decay)
    if name == "rmsprop":
        return torch.optim.RMSprop(params, lr=lr, momentum=momentum, weight_decay=weight_decay)
    raise ValueError(f"Unknown optimizer: {name}")

def train_one(config, run_dir):
    run_dir = Path(run_dir)
    run_dir.mkdir(parents=True, exist_ok=True)

    with open(run_dir/"config.json", "w") as f:
        json.dump(config, f, indent=2)

    # Model
    if config["pretrained"]:
        model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
    else:
        model = models.resnet50(weights=None)

    model.fc = nn.Linear(model.fc.in_features, len(class_names))
    model = model.to(device)

    # Loss
    criterion = nn.CrossEntropyLoss(label_smoothing=config["label_smoothing"])

    # Optimizer (swept)
    optimizer = make_optimizer(
        config["optimizer"],
        model.parameters(),
        lr=config["lr"],
        weight_decay=config["weight_decay"],
        momentum=config.get("momentum", 0.9),
    )

    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config["epochs"])
    scaler = torch.amp.GradScaler('cuda', enabled=(device == "cuda"))

    train_loader, test_loader = make_loaders(config["input_size"], batch_size=config.get("batch_size", 32))

    best_acc, best_epoch = 0.0, -1
    history = []

    def run_epoch(loader, train: bool):
        model.train(train)
        total_loss, correct, total = 0.0, 0, 0

        for x, y in loader:
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)

            with torch.set_grad_enabled(train):
                with torch.amp.autocast('cuda', enabled=(device == "cuda")):
                    logits = model(x)
                    loss = criterion(logits, y)

                if train:
                    optimizer.zero_grad(set_to_none=True)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()

            total_loss += float(loss.item()) * x.size(0)
            correct += int((logits.argmax(1) == y).sum().item())
            total += int(x.size(0))

        return total_loss / max(total, 1), correct / max(total, 1)

    last_path = run_dir / "last.pt"
    best_path = run_dir / "best.pt"

    for epoch in range(config["epochs"]):
        tr_loss, tr_acc = run_epoch(train_loader, True)
        va_loss, va_acc = run_epoch(test_loader, False)
        scheduler.step()

        row = {
            "epoch": epoch,
            "train_loss": tr_loss, "train_acc": tr_acc,
            "val_loss": va_loss, "val_acc": va_acc,
        }
        history.append(row)

        save_checkpoint(
            str(last_path),
            epoch=epoch, model=model, optimizer=optimizer,
            scheduler=scheduler, scaler=scaler,
            best_metric=best_acc, class_names=class_names,
            extra=row
        )

        if va_acc > best_acc:
            best_acc, best_epoch = va_acc, epoch
            save_checkpoint(
                str(best_path),
                epoch=epoch, model=model, optimizer=optimizer,
                scheduler=scheduler, scaler=scaler,
                best_metric=best_acc, class_names=class_names,
                extra=row
            )

    with open(run_dir/"history.json", "w") as f:
        json.dump(history, f, indent=2)

    return {
        "best_val_acc": best_acc,
        "best_epoch": best_epoch,
        "final_val_acc": history[-1]["val_acc"]
    }


Device: cuda


In [ ]:
import itertools, random
from pathlib import Path

sweep_dir = Path(SWEEP_DIR)
sweep_dir.mkdir(parents=True, exist_ok=True)

grid = {
    "pretrained": [False, True],
    "optimizer": ["sgd", "adamw"],
    "input_size": [450],
    "epochs": [30],
    "label_smoothing": [0.0, 0.1],
    "weight_decay": [1e-4, 5e-4, 0.01, 0.05],
    "lr": [1e-4, 3e-4, 1e-3, 0.01, 0.05, 0.1],
    "momentum": [0.9],
}

configs = [dict(zip(grid.keys(), vals)) for vals in itertools.product(*grid.values())]

# Prune combinations that are almost always bad
pruned = []
for c in configs:
    if c["pretrained"]:
        # Fine-tuning: mainly AdamW ; SGD can work
        if c["optimizer"] in ["adam", "adamw"] and c["lr"] > 3e-3:
            continue
        if c["optimizer"] == "sgd" and c["lr"] < 1e-3:
            continue
    else:
        # random initialization: mainly SGD prefers higher LR; AdamW can work
        if c["optimizer"] == "sgd" and c["lr"] < 1e-2:
            continue
        if c["optimizer"] in ["adam", "adamw"] and c["lr"] < 1e-3:
            continue
        if c["optimizer"] in ["adam", "adamw"] and c["lr"] > 3e-2:
            continue

    pruned.append(c)

configs = pruned
# random.shuffle(configs)

print("Num sweep configs:", len(configs))
configs[:5]


Num sweep configs: 96


[{'pretrained': False,
  'optimizer': 'sgd',
  'input_size': 450,
  'epochs': 30,
  'label_smoothing': 0.0,
  'weight_decay': 0.0001,
  'lr': 0.01,
  'momentum': 0.9},
 {'pretrained': False,
  'optimizer': 'sgd',
  'input_size': 450,
  'epochs': 30,
  'label_smoothing': 0.0,
  'weight_decay': 0.0001,
  'lr': 0.05,
  'momentum': 0.9},
 {'pretrained': False,
  'optimizer': 'sgd',
  'input_size': 450,
  'epochs': 30,
  'label_smoothing': 0.0,
  'weight_decay': 0.0001,
  'lr': 0.1,
  'momentum': 0.9},
 {'pretrained': False,
  'optimizer': 'sgd',
  'input_size': 450,
  'epochs': 30,
  'label_smoothing': 0.0,
  'weight_decay': 0.0005,
  'lr': 0.01,
  'momentum': 0.9},
 {'pretrained': False,
  'optimizer': 'sgd',
  'input_size': 450,
  'epochs': 30,
  'label_smoothing': 0.0,
  'weight_decay': 0.0005,
  'lr': 0.05,
  'momentum': 0.9}]

In [ ]:
import json, hashlib

def run_id_from_cfg(cfg):
    base = (
        f"exp"
        f"_pt{int(cfg['pretrained'])}"
        f"_{cfg['optimizer']}"
        f"_sz{cfg['input_size']}"
        f"_lr{cfg['lr']}"
        f"_wd{cfg['weight_decay']}"
        f"_ls{cfg['label_smoothing']}"
        f"_ep{cfg['epochs']}"
    )

    h = hashlib.md5(json.dumps(cfg, sort_keys=True).encode()).hexdigest()[:8]
    return f"{base}__{h}"

## Train and save

In [ ]:
import time
import pandas as pd

results_path = sweep_dir / "results.csv"

done = set()
if results_path.exists():
    df_prev = pd.read_csv(results_path)
    done = set(df_prev["run_id"].astype(str).tolist())
    print("Found", len(done), "completed runs in results.csv")
else:
    df_prev = pd.DataFrame()

rows = []

for i, cfg in enumerate(configs):
    run_id = run_id_from_cfg(cfg)

    if run_id in done:
        print(run_id, "=> already done")
        continue

    print(f"starting run {run_id}", end = "\r")
    run_dir = sweep_dir / run_id
    t0 = time.time()
    out = train_one(cfg, run_dir)
    dt = time.time() - t0

    row = {"run_id": run_id, **cfg, **out, "seconds": dt}
    rows.append(row)

    df_new = pd.DataFrame(rows)
    df_all = pd.concat([df_prev, df_new], ignore_index=True)
    df_all.to_csv(results_path, index=False)

    print(run_id, "=> best_val_acc:", out["best_val_acc"], "time(s):", int(dt))

print("Saved:", results_path)


Found 77 completed runs in results.csv
exp_pt0_sgd_sz450_lr0.01_wd0.0001_ls0.0_ep30__ff1dfe2c => best_val_acc: 0.44140625 time(s): 1198
exp_pt0_sgd_sz450_lr0.05_wd0.0001_ls0.0_ep30__6a1b7b56 => already done
exp_pt0_sgd_sz450_lr0.1_wd0.0001_ls0.0_ep30__f1b69046 => already done
exp_pt0_sgd_sz450_lr0.01_wd0.0005_ls0.0_ep30__874374a7 => already done
exp_pt0_sgd_sz450_lr0.05_wd0.0005_ls0.0_ep30__b3eea74d => already done
exp_pt0_sgd_sz450_lr0.1_wd0.0005_ls0.0_ep30__1b2859f4 => already done
exp_pt0_sgd_sz450_lr0.01_wd0.01_ls0.0_ep30__43a0a0bc => best_val_acc: 0.5 time(s): 709
exp_pt0_sgd_sz450_lr0.05_wd0.01_ls0.0_ep30__a51faa50 => already done
exp_pt0_sgd_sz450_lr0.1_wd0.01_ls0.0_ep30__c99b5e50 => already done
exp_pt0_sgd_sz450_lr0.01_wd0.05_ls0.0_ep30__6e42e563 => already done
exp_pt0_sgd_sz450_lr0.05_wd0.05_ls0.0_ep30__76d86f6d => already done
exp_pt0_sgd_sz450_lr0.1_wd0.05_ls0.0_ep30__cacd9ed1 => already done
exp_pt0_sgd_sz450_lr0.01_wd0.0001_ls0.1_ep30__ee297cdb => already done
exp_pt0_sg

100%|██████████| 97.8M/97.8M [00:00<00:00, 230MB/s]


exp_pt1_sgd_sz450_lr0.01_wd0.01_ls0.1_ep30__855efd19 => best_val_acc: 0.7734375 time(s): 712
exp_pt1_sgd_sz450_lr0.05_wd0.01_ls0.1_ep30__fad3538c => already done
exp_pt1_sgd_sz450_lr0.1_wd0.01_ls0.1_ep30__249889d3 => already done
exp_pt1_sgd_sz450_lr0.001_wd0.05_ls0.1_ep30__d578fb03 => best_val_acc: 0.7890625 time(s): 683
exp_pt1_sgd_sz450_lr0.01_wd0.05_ls0.1_ep30__4ca23f4b => best_val_acc: 0.39453125 time(s): 676
exp_pt1_sgd_sz450_lr0.05_wd0.05_ls0.1_ep30__b37aff92 => already done
exp_pt1_sgd_sz450_lr0.1_wd0.05_ls0.1_ep30__61f5f0aa => best_val_acc: 0.125 time(s): 599
exp_pt1_adamw_sz450_lr0.0001_wd0.0001_ls0.0_ep30__ae211220 => best_val_acc: 0.8125 time(s): 656
exp_pt1_adamw_sz450_lr0.0003_wd0.0001_ls0.0_ep30__f64d9442 => best_val_acc: 0.7734375 time(s): 667
exp_pt1_adamw_sz450_lr0.001_wd0.0001_ls0.0_ep30__401dc36b => already done
exp_pt1_adamw_sz450_lr0.0001_wd0.0005_ls0.0_ep30__5055498e => best_val_acc: 0.78515625 time(s): 662
exp_pt1_adamw_sz450_lr0.0003_wd0.0005_ls0.0_ep30__584b19

## Results

In [ ]:
import pandas as pd

df = pd.read_csv(results_path)

print("Top overall:")
display(df.sort_values("best_val_acc", ascending=False).head(15))

print("\nTop pretrained:")
display(df[df["pretrained"] == True].sort_values("best_val_acc", ascending=False).head(10))

print("\nTop scratch:")
display(df[df["pretrained"] == False].sort_values("best_val_acc", ascending=False).head(10))

print("\nTop by optimizer:")
for opt in sorted(df["optimizer"].unique()):
    print(f"\nOptimizer = {opt}")
    display(df[df["optimizer"] == opt].sort_values("best_val_acc", ascending=False).head(10))


Top overall:


,run_id,pretrained,optimizer,input_size,epochs,label_smoothing,weight_decay,lr,momentum,best_val_acc,best_epoch,final_val_acc,status,seconds
97,exp_pt1_adamw_sz450_lr0.0003_wd0.05_ls0.1_ep30...,True,adamw,450,30,0.1,0.0500,0.0003,0.9,0.851562,27.0,0.839844,NaN,680.518760
0,exp_pt1_adamw_sz450_lr0.0003_wd0.0005_ls0.1_ep...,True,adamw,450,30,0.1,0.0005,0.0003,0.9,0.843750,19.0,0.812500,ok,NaN
94,exp_pt1_adamw_sz450_lr0.0003_wd0.0001_ls0.1_ep...,True,adamw,450,30,0.1,0.0001,0.0003,0.9,0.832031,24.0,0.761719,NaN,674.756533
95,exp_pt1_adamw_sz450_lr0.0001_wd0.0005_ls0.1_ep...,True,adamw,450,30,0.1,0.0005,0.0001,0.9,0.832031,27.0,0.804688,NaN,593.204993
1,exp_pt1_adamw_sz450_lr0.0001_wd0.05_ls0.1_ep30...,True,adamw,450,30,0.1,0.0500,0.0001,0.9,0.832031,26.0,0.816406,ok,NaN
91,exp_pt1_adamw_sz450_lr0.0003_wd0.05_ls0.0_ep30...,True,adamw,450,30,0.0,0.0500,0.0003,0.9,0.828125,9.0,0.742188,NaN,632.668859
93,exp_pt1_adamw_sz450_lr0.0001_wd0.0001_ls0.1_ep...,True,adamw,450,30,0.1,0.0001,0.0001,0.9,0.828125,18.0,0.808594,NaN,660.745937
73,exp_pt1_sgd_sz450_lr0.01_wd0.0001_ls0.1_ep30__...,True,sgd,450,30,0.1,0.0001,0.0100,0.9,0.824219,28.0,0.808594,NaN,723.136463
2,exp_pt1_adamw_sz450_lr0.0001_wd0.01_ls0.0_ep30...,True,adamw,450,30,0.0,0.0100,0.0001,0.9,0.816406,21.0,0.800781,ok,NaN
90,exp_pt1_adamw_sz450_lr0.0001_wd0.05_ls0.0_ep30...,True,adamw,450,30,0.0,0.0500,0.0001,0.9,0.816406,10.0,0.785156,NaN,649.225539



Top pretrained:


,run_id,pretrained,optimizer,input_size,epochs,label_smoothing,weight_decay,lr,momentum,best_val_acc,best_epoch,final_val_acc,status,seconds
97,exp_pt1_adamw_sz450_lr0.0003_wd0.05_ls0.1_ep30...,True,adamw,450,30,0.1,0.0500,0.0003,0.9,0.851562,27.0,0.839844,NaN,680.518760
0,exp_pt1_adamw_sz450_lr0.0003_wd0.0005_ls0.1_ep...,True,adamw,450,30,0.1,0.0005,0.0003,0.9,0.843750,19.0,0.812500,ok,NaN
94,exp_pt1_adamw_sz450_lr0.0003_wd0.0001_ls0.1_ep...,True,adamw,450,30,0.1,0.0001,0.0003,0.9,0.832031,24.0,0.761719,NaN,674.756533
95,exp_pt1_adamw_sz450_lr0.0001_wd0.0005_ls0.1_ep...,True,adamw,450,30,0.1,0.0005,0.0001,0.9,0.832031,27.0,0.804688,NaN,593.204993
1,exp_pt1_adamw_sz450_lr0.0001_wd0.05_ls0.1_ep30...,True,adamw,450,30,0.1,0.0500,0.0001,0.9,0.832031,26.0,0.816406,ok,NaN
91,exp_pt1_adamw_sz450_lr0.0003_wd0.05_ls0.0_ep30...,True,adamw,450,30,0.0,0.0500,0.0003,0.9,0.828125,9.0,0.742188,NaN,632.668859
93,exp_pt1_adamw_sz450_lr0.0001_wd0.0001_ls0.1_ep...,True,adamw,450,30,0.1,0.0001,0.0001,0.9,0.828125,18.0,0.808594,NaN,660.745937
73,exp_pt1_sgd_sz450_lr0.01_wd0.0001_ls0.1_ep30__...,True,sgd,450,30,0.1,0.0001,0.0100,0.9,0.824219,28.0,0.808594,NaN,723.136463
2,exp_pt1_adamw_sz450_lr0.0001_wd0.01_ls0.0_ep30...,True,adamw,450,30,0.0,0.0100,0.0001,0.9,0.816406,21.0,0.800781,ok,NaN
90,exp_pt1_adamw_sz450_lr0.0001_wd0.05_ls0.0_ep30...,True,adamw,450,30,0.0,0.0500,0.0001,0.9,0.816406,10.0,0.785156,NaN,649.225539



Top scratch:


,run_id,pretrained,optimizer,input_size,epochs,label_smoothing,weight_decay,lr,momentum,best_val_acc,best_epoch,final_val_acc,status,seconds
58,exp_pt0_adamw_sz450_lr0.001_wd0.01_ls0.1_ep30_...,False,adamw,450,30,0.1,0.0100,0.001,0.9,0.578125,27.0,0.550781,NaN,706.376623
18,exp_pt0_adamw_sz450_lr0.001_wd0.05_ls0.0_ep30_...,False,adamw,450,30,0.0,0.0500,0.001,0.9,0.531250,27.0,0.500000,ok,NaN
60,exp_pt0_adamw_sz450_lr0.001_wd0.05_ls0.1_ep30_...,False,adamw,450,30,0.1,0.0500,0.001,0.9,0.531250,24.0,0.507812,NaN,717.972894
56,exp_pt0_adamw_sz450_lr0.001_wd0.0001_ls0.1_ep3...,False,adamw,450,30,0.1,0.0001,0.001,0.9,0.527344,28.0,0.519531,NaN,693.573740
51,exp_pt0_adamw_sz450_lr0.001_wd0.0005_ls0.0_ep3...,False,adamw,450,30,0.0,0.0005,0.001,0.9,0.527344,29.0,0.527344,NaN,725.240090
53,exp_pt0_adamw_sz450_lr0.001_wd0.01_ls0.0_ep30_...,False,adamw,450,30,0.0,0.0100,0.001,0.9,0.511719,29.0,0.511719,NaN,702.402009
19,exp_pt0_sgd_sz450_lr0.01_wd0.01_ls0.1_ep30__c3...,False,sgd,450,30,0.1,0.0100,0.010,0.9,0.503906,29.0,0.503906,ok,NaN
20,exp_pt0_sgd_sz450_lr0.01_wd0.0005_ls0.1_ep30__...,False,sgd,450,30,0.1,0.0005,0.010,0.9,0.500000,24.0,0.496094,ok,NaN
78,exp_pt0_sgd_sz450_lr0.01_wd0.01_ls0.0_ep30__43...,False,sgd,450,30,0.0,0.0100,0.010,0.9,0.500000,29.0,0.500000,NaN,709.889248
54,exp_pt0_adamw_sz450_lr0.01_wd0.01_ls0.0_ep30__...,False,adamw,450,30,0.0,0.0100,0.010,0.9,0.484375,29.0,0.484375,NaN,719.841771



Top by optimizer:

Optimizer = adamw


,run_id,pretrained,optimizer,input_size,epochs,label_smoothing,weight_decay,lr,momentum,best_val_acc,best_epoch,final_val_acc,status,seconds
97,exp_pt1_adamw_sz450_lr0.0003_wd0.05_ls0.1_ep30...,True,adamw,450,30,0.1,0.0500,0.0003,0.9,0.851562,27.0,0.839844,NaN,680.518760
0,exp_pt1_adamw_sz450_lr0.0003_wd0.0005_ls0.1_ep...,True,adamw,450,30,0.1,0.0005,0.0003,0.9,0.843750,19.0,0.812500,ok,NaN
94,exp_pt1_adamw_sz450_lr0.0003_wd0.0001_ls0.1_ep...,True,adamw,450,30,0.1,0.0001,0.0003,0.9,0.832031,24.0,0.761719,NaN,674.756533
1,exp_pt1_adamw_sz450_lr0.0001_wd0.05_ls0.1_ep30...,True,adamw,450,30,0.1,0.0500,0.0001,0.9,0.832031,26.0,0.816406,ok,NaN
95,exp_pt1_adamw_sz450_lr0.0001_wd0.0005_ls0.1_ep...,True,adamw,450,30,0.1,0.0005,0.0001,0.9,0.832031,27.0,0.804688,NaN,593.204993
91,exp_pt1_adamw_sz450_lr0.0003_wd0.05_ls0.0_ep30...,True,adamw,450,30,0.0,0.0500,0.0003,0.9,0.828125,9.0,0.742188,NaN,632.668859
93,exp_pt1_adamw_sz450_lr0.0001_wd0.0001_ls0.1_ep...,True,adamw,450,30,0.1,0.0001,0.0001,0.9,0.828125,18.0,0.808594,NaN,660.745937
2,exp_pt1_adamw_sz450_lr0.0001_wd0.01_ls0.0_ep30...,True,adamw,450,30,0.0,0.0100,0.0001,0.9,0.816406,21.0,0.800781,ok,NaN
90,exp_pt1_adamw_sz450_lr0.0001_wd0.05_ls0.0_ep30...,True,adamw,450,30,0.0,0.0500,0.0001,0.9,0.816406,10.0,0.785156,NaN,649.225539
87,exp_pt1_adamw_sz450_lr0.0003_wd0.0005_ls0.0_ep...,True,adamw,450,30,0.0,0.0005,0.0003,0.9,0.812500,8.0,0.757812,NaN,633.479165



Optimizer = sgd


,run_id,pretrained,optimizer,input_size,epochs,label_smoothing,weight_decay,lr,momentum,best_val_acc,best_epoch,final_val_acc,status,seconds
73,exp_pt1_sgd_sz450_lr0.01_wd0.0001_ls0.1_ep30__...,True,sgd,450,30,0.1,0.0001,0.010,0.9,0.824219,28.0,0.808594,NaN,723.136463
65,exp_pt1_sgd_sz450_lr0.01_wd0.0005_ls0.0_ep30__...,True,sgd,450,30,0.0,0.0005,0.010,0.9,0.812500,16.0,0.781250,NaN,707.999598
4,exp_pt1_sgd_sz450_lr0.05_wd0.0001_ls0.1_ep30__...,True,sgd,450,30,0.1,0.0001,0.050,0.9,0.800781,21.0,0.785156,ok,NaN
70,exp_pt1_sgd_sz450_lr0.001_wd0.05_ls0.0_ep30__6...,True,sgd,450,30,0.0,0.0500,0.001,0.9,0.796875,22.0,0.785156,NaN,742.446805
5,exp_pt1_sgd_sz450_lr0.001_wd0.0001_ls0.0_ep30_...,True,sgd,450,30,0.0,0.0001,0.001,0.9,0.792969,21.0,0.781250,ok,NaN
6,exp_pt1_sgd_sz450_lr0.01_wd0.0001_ls0.0_ep30__...,True,sgd,450,30,0.0,0.0001,0.010,0.9,0.789062,15.0,0.765625,ok,NaN
81,exp_pt1_sgd_sz450_lr0.001_wd0.05_ls0.1_ep30__d...,True,sgd,450,30,0.1,0.0500,0.001,0.9,0.789062,25.0,0.742188,NaN,683.841883
68,exp_pt1_sgd_sz450_lr0.001_wd0.01_ls0.0_ep30__6...,True,sgd,450,30,0.0,0.0100,0.001,0.9,0.785156,17.0,0.777344,NaN,716.355140
64,exp_pt1_sgd_sz450_lr0.001_wd0.0005_ls0.0_ep30_...,True,sgd,450,30,0.0,0.0005,0.001,0.9,0.773438,24.0,0.753906,NaN,721.089293
80,exp_pt1_sgd_sz450_lr0.01_wd0.01_ls0.1_ep30__85...,True,sgd,450,30,0.1,0.0100,0.010,0.9,0.773438,28.0,0.769531,NaN,712.878149
